In [1]:
!pip3 install -r requirements.txt
!pip3 install flask

Defaulting to user installation because normal site-packages is not writeable
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.2 requires click>=8.0, but you have click 7.1.2 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit

In [2]:


import cv2
from glob import glob
from pathlib import Path

from model import Model
from utils import plot

/var/data/python/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
model = Model()

# Read video 
### and save predictions for each frame as another video

In [4]:
def process_stream(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    prediction = model.predict(frame)
    label = prediction['label']
    conf = prediction['confidence']
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return label, conf, frame

In [5]:
from flask import Flask, render_template, Response, current_app
import flask
import socket
import cv2, time

notifications = {"0": "Unknown"}

        
app = Flask('hello')
def gen_frames_camera(camera_num):  
    camera = cv2.VideoCapture(camera_num)  # CAP_DSHOW because of https://answers.opencv.org/question/234933/opencv-440modulesvideoiosrccap_msmfcpp-682-cvcapture_msmfinitstream-failed-to-set-mediatype-stream-0-640x480-30-mfvideoformat_rgb24unsupported-media/
    while True:
        success, frame = camera.read()
        if not success:
            break
        else:
            label, conf, frame = process_stream(frame)
            print(label, conf)
            if (label == "fight on a street") | (label == "street violence") | (label == "violence in office"):
                notifications["0"] = label
                time.sleep(2)
                notification()
                


            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


def gen_frames_drone():
    tello_video = cv2.VideoCapture("udp://@0.0.0.0:11111")
    while True:
        try:
            ret, frame = tello_video.read()

            if ret:
                ret, buffer = cv2.imencode('.jpg', frame)
                frame = buffer.tobytes()
                yield (b'--frame\r\nContent-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

        except Exception as err:
            print(err)
            tello_video.release()
        
            # sock.sendto("ccw 22.5".encode(), tello_address)
            # response = sock.recv(1024).decode()
            # print("response #4", response)
            # time.sleep(0.5)



@app.route('/video_feed_0')
def video_feed_0():
    return Response(gen_frames_camera(0), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_1')
def video_feed_1():
    return Response(gen_frames_camera(2), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_2')
def video_feed_2():
    return Response(gen_frames_camera(6), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_drone')
def video_feed_drone():
    return Response(gen_frames_drone(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return flask.render_template('./index.html')

@app.route('/notification')
def notification():
    return flask.render_template('./notification.html', data=notifications)


app.run()

 * Serving Flask app 'hello'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Sep/2022 07:13:57] "GET /notification HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2022 07:13:58] "GET /video_feed_0 HTTP/1.1" 200 -


Unknown 0.22052732110023499
Unknown 0.22080305218696594
person walking in office 0.2351427674293518
person walking in office 0.23435217142105103
person walking in office 0.23080022633075714
Unknown 0.22491508722305298
person walking in office 0.23506398499011993
fire in office 0.23095010221004486
fire in office 0.23061244189739227
person walking in office 0.23793113231658936
fire in office 0.2429579347372055
person walking in office 0.23406557738780975
fire in office 0.24525637924671173
fire in office 0.23449574410915375
fire in office 0.2325727492570877
person walking in office 0.2360953539609909
fire in office 0.23388583958148956
Unknown 0.2258201241493225
person walking in office 0.2304772138595581
Unknown 0.22718581557273865
Unknown 0.22820134460926056
person walking in office 0.23869502544403076
Unknown 0.22874607145786285
fire in office 0.230185404419899
Unknown 0.22625884413719177
Unknown 0.22809143364429474
person walking in office 0.2325645387172699
Unknown 0.22559405863285065

Error on request:
Traceback (most recent call last):
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/serving.py", line 335, in run_wsgi
    execute(self.server.app)
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/serving.py", line 324, in execute
    for data in application_iter:
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/wsgi.py", line 500, in __next__
    return self._next()
  File "/var/data/python/lib/python3.9/site-packages/werkzeug/wrappers/response.py", line 50, in _iter_encoded
    for item in iterable:
  File "/tmp/ipykernel_5377/1996422697.py", line 22, in gen_frames_camera
    notification()
  File "/tmp/ipykernel_5377/1996422697.py", line 75, in notification
    return flask.render_template('./notification.html', data=notifications)
  File "/var/data/python/lib/python3.9/site-packages/flask/templating.py", line 145, in render_template
    app = current_app._get_current_object()  # type: ignore[attr-defined]
  File "/var/data/py

In [ ]:
# import the opencv library
import cv2
  
  
# define a video capture object
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    label, conf, frame = process_stream(frame)
    
    # Display the resulting frame
    cv2.imshow("feed", frame)
    print(label, conf)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()